In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2017 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2017"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3],NaN
1,J A N U A R Y,6,Arsenal,Lionsgate Premiere,Steven C. Miller (director); Jason Mosberg (sc...,[4][5],NaN
2,J A N U A R Y,6,Between Us,IFC Films / Nora Films,Rafael Palacio Illingworth (director/screenpla...,[6],NaN
3,J A N U A R Y,13,Monster Trucks,Paramount Pictures / Paramount Animation / Nic...,Chris Wedge (director); Derek Connolly (screen...,[7][8],NaN
4,J A N U A R Y,13,The Bye Bye Man,STX Entertainment / Intrepid Pictures,Stacy Title (director); Jonathan Penner (scree...,[9],NaN
...,...,...,...,...,...,...,...
242,D E C E M B E R,22,Crooked House,Vertical Entertainment / Sony Pictures,Gilles Paquet-Brenner (director/screenplay); T...,NaN,[293]
243,D E C E M B E R,22,Hostiles,Entertainment Studios,Scott Cooper (director/screenplay); Christian ...,NaN,[294]
244,D E C E M B E R,25,All the Money in the World,TriStar Pictures,Ridley Scott (director); David Scarpa (screenp...,NaN,[295]
245,D E C E M B E R,25,Molly's Game,STX Entertainment,Aaron Sorkin (director/screenplay); Jessica Ch...,NaN,[296]


In [10]:
df_2017 = df[['Title','Cast and crew']]

In [11]:
df_2017

,Title,Cast and crew
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...
2,Between Us,Rafael Palacio Illingworth (director/screenpla...
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...
...,...,...
242,Crooked House,Gilles Paquet-Brenner (director/screenplay); T...
243,Hostiles,Scott Cooper (director/screenplay); Christian ...
244,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...
245,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...


In [12]:
!pip install tmdbv3api

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '703a12952f3292e9958bc4d2d904a228'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2017['genres'] = df_2017['Title'].map(lambda x: get_genre(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\982775569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017['genres'] = df_2017['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2017

,Title,Cast and crew,genres
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...,Fantasy Action Thriller Horror
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...,Thriller Crime
2,Between Us,Rafael Palacio Illingworth (director/screenpla...,Drama
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...,Action Comedy Science Fiction
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...,Horror Thriller
...,...,...,...
242,Crooked House,Gilles Paquet-Brenner (director/screenplay); T...,Mystery Crime Thriller
243,Hostiles,Scott Cooper (director/screenplay); Christian ...,Western Drama History
244,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...,Thriller Drama Crime History
245,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...,Drama Crime


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2017['director_name'] = df_2017['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\1394242410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017['director_name'] = df_2017['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [20]:
df_2017['actor_1_name'] = df_2017['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\723709796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017['actor_1_name'] = df_2017['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [21]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [22]:
df_2017['actor_2_name'] = df_2017['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\2545097929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017['actor_2_name'] = df_2017['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [23]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [24]:
df_2017['actor_3_name'] = df_2017['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [25]:
df_2017

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...,Fantasy Action Thriller Horror,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...,Thriller Crime,Steven C. Miller,Adrian Grenier,Johnathon Schaech,Nicolas Cage
2,Between Us,Rafael Palacio Illingworth (director/screenpla...,Drama,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...,Action Comedy Science Fiction,Chris Wedge,Lucas Till,Jane Levy,Amy Ryan
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...,Horror Thriller,Stacy Title,Douglas Smith,Doug Jones,Michael Trucco
...,...,...,...,...,...,...,...
242,Crooked House,Gilles Paquet-Brenner (director/screenplay); T...,Mystery Crime Thriller,Gilles Paquet-Brenner,Max Irons,Glenn Close,Gillian Anderson
243,Hostiles,Scott Cooper (director/screenplay); Christian ...,Western Drama History,Scott Cooper,Christian Bale,Rosamund Pike,Wes Studi
244,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...,Thriller Drama Crime History,Ridley Scott,Michelle Williams,Mark Wahlberg,Christopher Plummer
245,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...,Drama Crime,Aaron Sorkin,Jessica Chastain,Idris Elba,Kevin Costner


In [26]:
df_2017 = df_2017.rename(columns={'Title':'movie_title'})

In [27]:
new_df17 = df_2017.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [28]:
new_df17

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver,Fantasy Action Thriller Horror,Underworld: Blood Wars
1,Steven C. Miller,Adrian Grenier,Johnathon Schaech,Nicolas Cage,Thriller Crime,Arsenal
2,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg,Drama,Between Us
3,Chris Wedge,Lucas Till,Jane Levy,Amy Ryan,Action Comedy Science Fiction,Monster Trucks
4,Stacy Title,Douglas Smith,Doug Jones,Michael Trucco,Horror Thriller,The Bye Bye Man
...,...,...,...,...,...,...
242,Gilles Paquet-Brenner,Max Irons,Glenn Close,Gillian Anderson,Mystery Crime Thriller,Crooked House
243,Scott Cooper,Christian Bale,Rosamund Pike,Wes Studi,Western Drama History,Hostiles
244,Ridley Scott,Michelle Williams,Mark Wahlberg,Christopher Plummer,Thriller Drama Crime History,All the Money in the World
245,Aaron Sorkin,Jessica Chastain,Idris Elba,Kevin Costner,Drama Crime,Molly's Game


In [29]:
new_df17['comb'] = new_df17['actor_1_name'] + ' ' + new_df17['actor_2_name'] + ' '+ new_df17['actor_3_name'] + ' '+ new_df17['director_name'] +' ' + new_df17['genres']

In [30]:
new_df17.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      4
actor_3_name     14
genres            0
movie_title       0
comb             14
dtype: int64

In [31]:
new_df17 = new_df17.dropna(how='any')

In [32]:
new_df17.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df17['movie_title'] = new_df17['movie_title'].str.lower()

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\898406936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df17['movie_title'] = new_df17['movie_title'].str.lower()


In [34]:
new_df17

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver,Fantasy Action Thriller Horror,underworld: blood wars,Kate Beckinsale Theo James Lara Pulver Anna Fo...
1,Steven C. Miller,Adrian Grenier,Johnathon Schaech,Nicolas Cage,Thriller Crime,arsenal,Adrian Grenier Johnathon Schaech Nicolas Cage ...
2,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg,Drama,between us,Olivia Thirlby Ben Feldman Adam Goldberg Rafae...
3,Chris Wedge,Lucas Till,Jane Levy,Amy Ryan,Action Comedy Science Fiction,monster trucks,Lucas Till Jane Levy Amy Ryan Chris Wedge Acti...
4,Stacy Title,Douglas Smith,Doug Jones,Michael Trucco,Horror Thriller,the bye bye man,Douglas Smith Doug Jones Michael Trucco Stacy ...
...,...,...,...,...,...,...,...
242,Gilles Paquet-Brenner,Max Irons,Glenn Close,Gillian Anderson,Mystery Crime Thriller,crooked house,Max Irons Glenn Close Gillian Anderson Gilles ...
243,Scott Cooper,Christian Bale,Rosamund Pike,Wes Studi,Western Drama History,hostiles,Christian Bale Rosamund Pike Wes Studi Scott C...
244,Ridley Scott,Michelle Williams,Mark Wahlberg,Christopher Plummer,Thriller Drama Crime History,all the money in the world,Michelle Williams Mark Wahlberg Christopher Pl...
245,Aaron Sorkin,Jessica Chastain,Idris Elba,Kevin Costner,Drama Crime,molly's game,Jessica Chastain Idris Elba Kevin Costner Aaro...


In [35]:
old_df = pd.read_csv('final_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [37]:
final_df = old_df.append(new_df17,ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_11876\2833379201.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df17,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6097,Gilles Paquet-Brenner,Max Irons,Glenn Close,Gillian Anderson,Mystery Crime Thriller,crooked house,Max Irons Glenn Close Gillian Anderson Gilles ...
6098,Scott Cooper,Christian Bale,Rosamund Pike,Wes Studi,Western Drama History,hostiles,Christian Bale Rosamund Pike Wes Studi Scott C...
6099,Ridley Scott,Michelle Williams,Mark Wahlberg,Christopher Plummer,Thriller Drama Crime History,all the money in the world,Michelle Williams Mark Wahlberg Christopher Pl...
6100,Aaron Sorkin,Jessica Chastain,Idris Elba,Kevin Costner,Drama Crime,molly's game,Jessica Chastain Idris Elba Kevin Costner Aaro...


In [39]:
final_df.to_csv('main_data.csv',index=False)